In [ ]:
#dependencies
import requests
import json


In [ ]:
#url for GET requests to retrive

In [ ]:
#create a scatter plot the shows the relationship between the below
Temperature (F) vs. Latitude

In [ ]:
#After each plot, add a sentence or two explaining what the code is analyzing.

In [ ]:
#create a scatter plot the shows the relationship between the below
Humidity (%) vs. Latitude

In [ ]:
#After each plot, add a sentence or two explaining what the code is analyzing.

In [ ]:
#create a scatter plot the shows the relationship between the below
Cloudiness (%) vs. Latitude

In [ ]:
#After each plot, add a sentence or two explaining what the code is analyzing.

In [ ]:
#create a scatter plot the shows the relationship between the below
Wind Speed (mph) vs. Latitude

In [ ]:
#After each plot, add a sentence or two explaining what the code is analyzing.

In [ ]:
#The second requirement is to run linear regression on each relationship. This time, separate the plots into Northern Hemisphere (greater than or equal to 0 degrees latitude) and Southern Hemisphere (less than 0 degrees latitude):

Northern Hemisphere - Temperature (F) vs. Latitude
Southern Hemisphere - Temperature (F) vs. Latitude
Northern Hemisphere - Humidity (%) vs. Latitude
Southern Hemisphere - Humidity (%) vs. Latitude
Northern Hemisphere - Cloudiness (%) vs. Latitude
Southern Hemisphere - Cloudiness (%) vs. Latitude
Northern Hemisphere - Wind Speed (mph) vs. Latitude
Southern Hemisphere - Wind Speed (mph) vs. Latitude

